#  Missing Value Imputation & Data Preprocessing Pipeline

This notebook demonstrates **different strategies to handle missing values** and **feature preprocessing** using Scikit-learn.
It covers **KNN Imputation**, **Iterative Imputation**, **Categorical Encoding**, and **Pipeline construction** for real-world datasets.

## Objective

- Understand missing value handling techniques
- Apply KNN and Iterative Imputation
- Encode categorical variables properly
- Build a clean preprocessing pipeline
- Prepare data for machine learning models

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
import pandas as pd

### Creating a Sample Dataset

We start with a **small synthetic dataset** containing:
- Numerical features: `height`, `weight`
- Categorical feature: `gender`

This dataset is used to **demonstrate imputation techniques clearly**.


In [ ]:
cls_data = pd.DataFrame({'height': [180, 165, 170, 185, 160, 175],
        'weight': [80, 55, 65, 90, 50, 75],
        'gender': ['Male', 'Female', 'Female', 'Male', 'Female', 'Male']})
cls_data

In [ ]:
import numpy as np

### Introducing Missing Values
- Missing values are intentionally introduced into the dataset to simulate
real-world data issues and observe how different imputers behave.


In [ ]:
cls_data.iloc[2, 1] = np.nan

In [ ]:
cls_data

In [ ]:
knn_imputer = KNNImputer().set_output(transform = 'pandas')
knn_imputer

In [ ]:
knn_imputer.fit_transform(cls_data[['height', 'weight']])

In [ ]:
cls_data = pd.DataFrame({'height': [180, 165, 170, 185, 160, 175],
        'weight': [80, 55, 65, 90, 50, 75],
        'gender': ['Male', 'Female', 'Female', 'Male', 'Female', 'Male']})
cls_data

In [ ]:
cls_data.iloc[3, 0]=np.nan

In [ ]:
cls_data

###  KNN Imputation (Numerical Features)

**KNN Imputer** replaces missing values by averaging values of the
nearest neighbors based on feature similarity.

This method works best when:
- Data is numeric
- Features are on similar scales


In [ ]:
knn_imputer.fit_transform(cls_data[['height','weight']])

### Encoding Categorical Variables

Machine learning models require numerical inputs.
The categorical variable `gender` is encoded into numerical values:

- Male → 0
- Female → 1


In [ ]:
cls_data['gender'] = cls_data['gender'].map({'Male': 0,'Female':1})
cls_data

In [ ]:
cls_data.iloc[2, 2] = np.nan

In [ ]:
cls_data

In [ ]:
knn_imputer.fit_transform(cls_data)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

###  Iterative Imputation

**Iterative Imputer** models each feature with missing values as a function
of other features and iteratively predicts missing entries.

Advantages:
- Handles complex feature relationships
- Works well for mixed data types


In [ ]:
iter_imputer = IterativeImputer(estimator= DecisionTreeClassifier(),
                                max_iter=100,
                                initial_strategy='most_frequent').set_output(transform = 'pandas')
iter_imputer

In [ ]:
iter_imputer.fit_transform(cls_data)

### Loading the Real Dataset

A real-world dataset is loaded to apply preprocessing techniques
on practical data and prepare it for machine learning.

In [ ]:
data = pd.read_csv(r"C:\Users\sande\Downloads\credit_approval_uci.csv")

In [ ]:
data.head()

In [ ]:
data.shape

### Feature–Target Separation

- `X` → Input features
- `y` → Target variable

This separation is required before model training.


In [ ]:
# Segregate data into X, y
X = data.drop('target', axis = 1)
y = data['target']

### Train–Test Split

The dataset is split into training and testing sets using **stratified sampling**
to preserve class distribution in the target variable.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

###  Identifying Categorical Features

Categorical columns are detected automatically and prepared
for encoding using `OrdinalEncoder`.


In [ ]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

In [ ]:
le =OrdinalEncoder()
le

In [ ]:
X_train.info()

In [ ]:
X_train.select_dtypes('object')

In [ ]:
cat_col = X_train.select_dtypes('object').columns
cat_col

In [ ]:
from sklearn.compose import ColumnTransformer

###  ColumnTransformer for Encoding

`ColumnTransformer` allows applying different transformations
to different columns while keeping the dataset structure intact.


In [ ]:
ct = ColumnTransformer([('Cat_enc', le, cat_col)],
                       remainder = 'passthrough',
                       verbose_feature_names_out= False).set_output(transform='pandas')
ct

In [ ]:
ct.fit_transform(X_train)

In [ ]:
X_train.select_dtypes('float64')

In [ ]:
num_col = X_train.select_dtypes('float64','int64').columns
num_col

In [ ]:
cat_col

In [ ]:
X_train = ct.fit_transform(X_train)
X_train.head()

### Advanced Imputation Strategy

- **Categorical features** → Iterative Imputer with RandomForestClassifier
- **Numerical features** → Iterative Imputer with RandomForestRegressor

This improves imputation quality by respecting feature types.


In [ ]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [ ]:
iter_imputer_cat = IterativeImputer(estimator = RandomForestClassifier(),
                                    max_iter = 100,
                                    initial_strategy = 'most_frequent')
iter_imputer_cat

In [ ]:
iter_imputer_num = IterativeImputer(estimator = RandomForestRegressor(),
                                    max_iter = 100,
                                    initial_strategy = 'mean')
iter_imputer_num

In [ ]:
ct2 = ColumnTransformer([('cat_imp', iter_imputer_cat, cat_col),
                         ('num_imp', iter_imputer_num, num_col)],
                        remainder = 'passthrough',
                        verbose_feature_names_out=False).set_output(transform = 'pandas')
ct2

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
pl = Pipeline([('ct1',ct),('ct2',ct2)])
pl

In [ ]:
X_test_transformed = pl.fit_transform(X_test)
X_test_transformed.head()